<h3>Big Data - Ex3</h3>
<h3>Avlas Kfir</h3>

<h3>term frequency–inverse document frequency</h3>


In [1]:
import pandas as pd
import math
import collections
from collections import Counter
import operator
import random


pd.set_option("display.max_colwidth", 1000)

In [2]:
# Read data

path = "/home/spark-vm/PycharmProjects/BigDataCource/"
folder_name = "BBC_News"

In [3]:
# Read text files

def read_directory(p_category: str):
    return sc.wholeTextFiles(path + "/" + folder_name + "/" + p_category)

In [4]:
def set_file_name(t):
    file_name = t[0]
    text = t[1]
    
    new_file_name = file_name[file_name.index(folder_name) + len(folder_name) + 1:]
    new_file_name = new_file_name.replace("/", "_")
    
    return (new_file_name, text)

In [5]:
def get_data():
    d1 = read_directory("bsns").map(set_file_name)
    d2 = read_directory("entrtmnt").map(set_file_name)
    d3 = read_directory("pltcs").map(set_file_name)
    d4 = read_directory("sprt").map(set_file_name)
    d5 = read_directory("tech").map(set_file_name)
    
    return d1.union(d2).union(d3).union(d4).union(d5)

In [6]:
main_data = get_data()

In [7]:
main_data.take(1)

[('bsns_382.txt',
  'Ban on forced retirement under 65\n\nEmployers will no longer be able to force workers to retire before 65, unless they can justify it.\n\nThe government has announced that firms will be barred from 2006 from imposing arbitrary retirement ages. Under new European age discrimination rules, a default retirement age of 65 will be introduced. Workers will be permitted to request staying on beyond this compulsory retirement age, although employers will have the right to refuse. Trade and Industry Secretary Patricia Hewitt said people would not be forced to work longer than they wanted, saying the default age was not a statutory, compulsory retirement age. She said employers would be free to continue employing people for as long as they were competent.\n\nUnder age discrimination proposals from the Department of Trade and Industry last year workers were to be allowed to work on till 70 if they wished.\n\nBusiness leaders had opposed the plan as they said it would be too 

In [8]:
num_of_documents = main_data.count()

In [10]:
num_of_documents

2225

In [9]:
def text_to_vec(t):
    
    file_name = t[0]
    text = t[1]
    words= [] 
    
    for txt in text.split("\n"):       # Text to sentences
        for word in txt.split(' '):    # Sentence to words
            words.append(word.lower()) # Word to lower case
    
    return (file_name, words)

In [16]:
data = main_data.map(text_to_vec)

In [17]:
data.take(1)

[('bsns_382.txt',
  ['ban',
   'on',
   'forced',
   'retirement',
   'under',
   '65',
   '',
   'employers',
   'will',
   'no',
   'longer',
   'be',
   'able',
   'to',
   'force',
   'workers',
   'to',
   'retire',
   'before',
   '65,',
   'unless',
   'they',
   'can',
   'justify',
   'it.',
   '',
   'the',
   'government',
   'has',
   'announced',
   'that',
   'firms',
   'will',
   'be',
   'barred',
   'from',
   '2006',
   'from',
   'imposing',
   'arbitrary',
   'retirement',
   'ages.',
   'under',
   'new',
   'european',
   'age',
   'discrimination',
   'rules,',
   'a',
   'default',
   'retirement',
   'age',
   'of',
   '65',
   'will',
   'be',
   'introduced.',
   'workers',
   'will',
   'be',
   'permitted',
   'to',
   'request',
   'staying',
   'on',
   'beyond',
   'this',
   'compulsory',
   'retirement',
   'age,',
   'although',
   'employers',
   'will',
   'have',
   'the',
   'right',
   'to',
   'refuse.',
   'trade',
   'and',
   'industry',
   

In [18]:
from stop_words import get_stop_words

stop_words = get_stop_words('en')

broadcast_sw = sc.broadcast(stop_words)

In [19]:
broadcast_sw.value

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 "can't",
 'cannot',
 'could',
 "couldn't",
 'did',
 "didn't",
 'do',
 'does',
 "doesn't",
 'doing',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 "hadn't",
 'has',
 "hasn't",
 'have',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 "here's",
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 "how's",
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 "let's",
 'me',
 'more',
 'most',
 "mustn't",
 'my',
 'myself',
 'no',
 'nor',
 'not',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'ought',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'same',
 "shan't",
 'she',
 "she'd",
 "she'll",
 "she's",
 'should',
 "s

In [20]:
def remove_stop_words(d):
    
    file_name = d[0]
    words = d[1]
    w = []
    
    for word in words:
        if word not in broadcast_sw.value:
            w.append(word)
    
    return (file_name , w)

In [21]:
data1 = data.map(remove_stop_words)

In [22]:
data1.take(1)

[('bsns_382.txt',
  ['ban',
   'forced',
   'retirement',
   '65',
   '',
   'employers',
   'will',
   'longer',
   'able',
   'force',
   'workers',
   'retire',
   '65,',
   'unless',
   'can',
   'justify',
   'it.',
   '',
   'government',
   'announced',
   'firms',
   'will',
   'barred',
   '2006',
   'imposing',
   'arbitrary',
   'retirement',
   'ages.',
   'new',
   'european',
   'age',
   'discrimination',
   'rules,',
   'default',
   'retirement',
   'age',
   '65',
   'will',
   'introduced.',
   'workers',
   'will',
   'permitted',
   'request',
   'staying',
   'beyond',
   'compulsory',
   'retirement',
   'age,',
   'although',
   'employers',
   'will',
   'right',
   'refuse.',
   'trade',
   'industry',
   'secretary',
   'patricia',
   'hewitt',
   'said',
   'people',
   'forced',
   'work',
   'longer',
   'wanted,',
   'saying',
   'default',
   'age',
   'statutory,',
   'compulsory',
   'retirement',
   'age.',
   'said',
   'employers',
   'free',
   'co

In [23]:
import re 

def remove_symbols(t):
    file_name = t[0]
    words = t[1]
    w = []
    
    for i in range(len(words)):
        word = re.sub("[^a-zA-Z\s]", ' ', words[i]).replace(' ','')
        if word:
            w.append(word)
    
    return (file_name , w)

In [24]:
documents = data1.map(remove_symbols)

In [25]:
documents.take(1)

[('bsns_382.txt',
  ['ban',
   'forced',
   'retirement',
   'employers',
   'will',
   'longer',
   'able',
   'force',
   'workers',
   'retire',
   'unless',
   'can',
   'justify',
   'it',
   'government',
   'announced',
   'firms',
   'will',
   'barred',
   'imposing',
   'arbitrary',
   'retirement',
   'ages',
   'new',
   'european',
   'age',
   'discrimination',
   'rules',
   'default',
   'retirement',
   'age',
   'will',
   'introduced',
   'workers',
   'will',
   'permitted',
   'request',
   'staying',
   'beyond',
   'compulsory',
   'retirement',
   'age',
   'although',
   'employers',
   'will',
   'right',
   'refuse',
   'trade',
   'industry',
   'secretary',
   'patricia',
   'hewitt',
   'said',
   'people',
   'forced',
   'work',
   'longer',
   'wanted',
   'saying',
   'default',
   'age',
   'statutory',
   'compulsory',
   'retirement',
   'age',
   'said',
   'employers',
   'free',
   'continue',
   'employing',
   'people',
   'long',
   'competent

In [26]:
def invert(d):
    
    doc   = d[0]    
    terms = d[1] 
    
    return [(t,[doc]) for t in terms]

In [27]:
# Invert from doc with [terms] to term with [docs]

terms = documents.flatMap(invert).reduceByKey(lambda x,y: x+y)

In [28]:
terms.take(1)

[('retirement',
  ['bsns_382.txt',
   'bsns_382.txt',
   'bsns_382.txt',
   'bsns_382.txt',
   'bsns_382.txt',
   'bsns_382.txt',
   'bsns_238.txt',
   'bsns_326.txt',
   'bsns_326.txt',
   'bsns_249.txt',
   'bsns_249.txt',
   'bsns_368.txt',
   'bsns_449.txt',
   'bsns_090.txt',
   'bsns_133.txt',
   'bsns_255.txt',
   'bsns_255.txt',
   'bsns_255.txt',
   'bsns_255.txt',
   'bsns_041.txt',
   'bsns_041.txt',
   'bsns_452.txt',
   'entrtmnt_048.txt',
   'entrtmnt_177.txt',
   'entrtmnt_044.txt',
   'entrtmnt_269.txt',
   'entrtmnt_269.txt',
   'pltcs_380.txt',
   'pltcs_042.txt',
   'pltcs_042.txt',
   'pltcs_132.txt',
   'pltcs_132.txt',
   'pltcs_132.txt',
   'pltcs_132.txt',
   'pltcs_132.txt',
   'pltcs_357.txt',
   'pltcs_406.txt',
   'pltcs_294.txt',
   'pltcs_089.txt',
   'pltcs_089.txt',
   'pltcs_156.txt',
   'pltcs_159.txt',
   'pltcs_317.txt',
   'sprt_437.txt',
   'sprt_241.txt',
   'sprt_462.txt',
   'sprt_347.txt',
   'sprt_347.txt',
   'sprt_413.txt',
   'sprt_367.txt'

In [29]:
# Build an Inverted Index structure:

# With posting_list = Dict{doc_name:tf of term in doc}

# II format is (term, (df of term in corpus, posting_list) 

def inverted_index(t):
    
    term = t[0]
    docs = dict(Counter(t[1]))
                
    dft = len(docs)
                
    return (term, (dft, docs))

In [30]:
inverted_index = terms.map(inverted_index)

In [31]:
inverted_index.take(1)

[('retirement',
  (37,
   {'bsns_382.txt': 6,
    'bsns_238.txt': 1,
    'bsns_326.txt': 2,
    'bsns_249.txt': 2,
    'bsns_368.txt': 1,
    'bsns_449.txt': 1,
    'bsns_090.txt': 1,
    'bsns_133.txt': 1,
    'bsns_255.txt': 4,
    'bsns_041.txt': 2,
    'bsns_452.txt': 1,
    'entrtmnt_048.txt': 1,
    'entrtmnt_177.txt': 1,
    'entrtmnt_044.txt': 1,
    'entrtmnt_269.txt': 2,
    'pltcs_380.txt': 1,
    'pltcs_042.txt': 2,
    'pltcs_132.txt': 5,
    'pltcs_357.txt': 1,
    'pltcs_406.txt': 1,
    'pltcs_294.txt': 1,
    'pltcs_089.txt': 2,
    'pltcs_156.txt': 1,
    'pltcs_159.txt': 1,
    'pltcs_317.txt': 1,
    'sprt_437.txt': 1,
    'sprt_241.txt': 1,
    'sprt_462.txt': 1,
    'sprt_347.txt': 2,
    'sprt_413.txt': 1,
    'sprt_367.txt': 1,
    'sprt_431.txt': 1,
    'sprt_361.txt': 1,
    'sprt_115.txt': 1,
    'sprt_062.txt': 1,
    'sprt_375.txt': 1,
    'sprt_460.txt': 2}))]

In [32]:
inverted_indexMap = inverted_index.collectAsMap()

In [33]:
broadcast_inverted_index = sc.broadcast(inverted_indexMap)

In [35]:
inv_ind = inverted_index.map(lambda t:(t[0], list(t[1][1].keys())))

In [36]:
cols = ["Term", "Docs"]
inv_ind = inv_ind.toDF(cols)
inv_ind.toPandas()

,Term,Docs
0,retirement,"[bsns_382.txt, bsns_238.txt, bsns_326.txt, bsns_249.txt, bsns_368.txt, bsns_449.txt, bsns_090.txt, bsns_133.txt, bsns_255.txt, bsns_041.txt, bsns_452.txt, entrtmnt_048.txt, entrtmnt_177.txt, entrtmnt_044.txt, entrtmnt_269.txt, pltcs_380.txt, pltcs_042.txt, pltcs_132.txt, pltcs_357.txt, pltcs_406.txt, pltcs_294.txt, pltcs_089.txt, pltcs_156.txt, pltcs_159.txt, pltcs_317.txt, sprt_437.txt, sprt_241.txt, sprt_462.txt, sprt_347.txt, sprt_413.txt, sprt_367.txt, sprt_431.txt, sprt_361.txt, sprt_115.txt, sprt_062.txt, sprt_375.txt, sprt_460.txt]"
1,employers,"[bsns_382.txt, bsns_238.txt, bsns_079.txt, bsns_146.txt, bsns_336.txt, entrtmnt_366.txt, pltcs_050.txt, pltcs_237.txt, pltcs_380.txt, pltcs_080.txt, pltcs_132.txt, pltcs_001.txt, pltcs_108.txt, pltcs_175.txt, pltcs_003.txt, pltcs_020.txt, pltcs_128.txt, pltcs_291.txt, tech_079.txt, tech_102.txt, tech_373.txt]"
2,workers,"[bsns_382.txt, bsns_272.txt, bsns_123.txt, bsns_022.txt, bsns_345.txt, bsns_034.txt, bsns_296.txt, bsns_029.txt, bsns_334.txt, bsns_061.txt, bsns_160.txt, bsns_149.txt, bsns_482.txt, bsns_253.txt, bsns_305.txt, bsns_183.txt, bsns_274.txt, bsns_416.txt, bsns_258.txt, bsns_284.txt, bsns_465.txt, bsns_235.txt, bsns_368.txt, bsns_387.txt, bsns_007.txt, bsns_292.txt, bsns_133.txt, bsns_162.txt, bsns_020.txt, bsns_087.txt, bsns_130.txt, bsns_275.txt, bsns_068.txt, bsns_287.txt, bsns_251.txt, bsns_356.txt, bsns_100.txt, bsns_442.txt, bsns_086.txt, bsns_117.txt, pltcs_022.txt, pltcs_388.txt, pltcs_359.txt, pltcs_237.txt, pltcs_380.txt, pltcs_042.txt, pltcs_365.txt, pltcs_289.txt, pltcs_357.txt, pltcs_069.txt, pltcs_400.txt, pltcs_005.txt, pltcs_063.txt, pltcs_312.txt, pltcs_250.txt, pltcs_108.txt, pltcs_162.txt, pltcs_294.txt, pltcs_311.txt, pltcs_393.txt, pltcs_291.txt, pltcs_373.txt, sprt_493.txt, tech_065.txt, tech_119.txt, tech_331.txt, tech_151.txt, tech_012.txt, tech_041.txt, tech_22..."
3,retire,"[bsns_382.txt, bsns_238.txt, bsns_227.txt, entrtmnt_016.txt, entrtmnt_117.txt, pltcs_045.txt, sprt_057.txt, sprt_484.txt, sprt_185.txt, sprt_413.txt, sprt_367.txt, sprt_028.txt, sprt_500.txt, sprt_031.txt, sprt_144.txt, sprt_459.txt, sprt_156.txt, sprt_480.txt, sprt_421.txt]"
4,government,"[bsns_382.txt, bsns_272.txt, bsns_491.txt, bsns_110.txt, bsns_246.txt, bsns_322.txt, bsns_245.txt, bsns_433.txt, bsns_437.txt, bsns_339.txt, bsns_016.txt, bsns_195.txt, bsns_411.txt, bsns_303.txt, bsns_409.txt, bsns_248.txt, bsns_286.txt, bsns_023.txt, bsns_478.txt, bsns_241.txt, bsns_200.txt, bsns_187.txt, bsns_428.txt, bsns_083.txt, bsns_178.txt, bsns_104.txt, bsns_328.txt, bsns_308.txt, bsns_022.txt, bsns_489.txt, bsns_266.txt, bsns_347.txt, bsns_501.txt, bsns_045.txt, bsns_161.txt, bsns_493.txt, bsns_277.txt, bsns_502.txt, bsns_388.txt, bsns_096.txt, bsns_093.txt, bsns_029.txt, bsns_498.txt, bsns_326.txt, bsns_487.txt, bsns_432.txt, bsns_380.txt, bsns_139.txt, bsns_260.txt, bsns_199.txt, bsns_048.txt, bsns_163.txt, bsns_164.txt, bsns_445.txt, bsns_426.txt, bsns_344.txt, bsns_072.txt, bsns_183.txt, bsns_273.txt, bsns_497.txt, bsns_025.txt, bsns_033.txt, bsns_231.txt, bsns_340.txt, bsns_120.txt, bsns_174.txt, bsns_389.txt, bsns_055.txt, bsns_299.txt, bsns_204.txt, bsns_257.txt, b..."
...,...,...
31526,siliconbased,[tech_329.txt]
31527,citizenbased,[tech_010.txt]
31528,autolinks,[tech_010.txt]
31529,unfeasible,[tech_152.txt]


In [37]:
def tf(d):
    
    doc   = d[0]
    terms = Counter(d[1])
   
    return(doc, list(terms.keys()), list(terms.values()))

In [38]:
tf = documents.map(tf)

In [39]:
cols = ["Doc", "Terms", "TF(t,d)"]
tf.toDF(cols).toPandas()

,Doc,Terms,"TF(t,d)"
0,bsns_382.txt,"[ban, forced, retirement, employers, will, longer, able, force, workers, retire, unless, can, justify, it, government, announced, firms, barred, imposing, arbitrary, ages, new, european, age, discrimination, rules, default, introduced, permitted, request, staying, beyond, compulsory, although, right, refuse, trade, industry, secretary, patricia, hewitt, said, people, work, wanted, saying, statutory, free, continue, employing, long, competent, proposals, department, last, year, allowed, till, wished, business, leaders, opposed, plan, costly, cumbersome, british, chambers, commerce, welcomed, latest, proposal, this, move, today, best, worlds, ability, define, end, point, employeremployee, relationship, employees, flexibility, past, concern, cowardly, complete, uturn, makes, mockery, governments, socalled, commitment, outlawing, ageism, leaving, incoming, law, unravel, ...]","[1, 2, 6, 4, 7, 2, 1, 1, 4, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 12, 3, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 7, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, ...]"
1,bsns_050.txt,"[bad, weather, hits, nestle, sales, combination, rising, raw, material, costs, sluggish, european, economy, hit, swiss, food, drink, giant, revenue, dipped, bn, francs, ice, cream, mineral, water, dampened, wet, summer, however, nestles, profits, margins, helped, strong, performance, americas, china, raise, dividend, paying, back, debt, said, strength, franc, us, dollar, disposal, businesses, challenging, trading, conditions, europe, dented, poor, across, continent, contrast, prolonged, heat, wave, severely, affected, demand, bottled, also, fell, although, chocolate, coffee, frozen, goods, petcare, products, performed, better, elsewhere, enjoyed, exceptional, year, north, america, outperforming, market, terms, growth, added, strongly, africa, asia, despite, impact, high, oil, prices, political, instability, total, earnings, ...]","[2, 2, 1, 10, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 4, 2, 2, 2, 2, 3, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, ...]"
2,bsns_057.txt,"[economy, stronger, forecast, uk, probably, grew, faster, rate, third, quarter, reported, according, bank, england, deputy, governor, rachel, lomax, private, sector, business, surveys, suggest, official, estimates, ms, said, collectively, show, rapid, slowdown, house, price, growth, pointed, out, means, despite, strong, economic, base, rates, will, stay, hold, data, comes, office, national, statistics, ons, though, reliable, takes, longer, publish, now, boe, calling, delivery, can, make, effective, policy, decisions, recent, work, shown, add, value, even, preliminary, available, speech, north, wales, club, due, second, estimate, friday, the, mpc, judges, overall, little, higher, currently, indicate, successful, monetary, depends, good, information, cited, late, s, example, time, weak, ...]","[4, 2, 1, 2, 2, 1, 2, 1, 3, 3, 1, 1, 4, 2, 1, 1, 2, 7, 2, 2, 2, 3, 1, 3, 2, 5, 8, 1, 1, 1, 3, 2, 1, 4, 1, 1, 1, 1, 1, 5, 1, 3, 2, 1, 1, 5, 1, 1, 2, 2, 5, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 4, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 5, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, ...]"
3,bsns_272.txt,"[markets, signal, brazilian, recovery, stock, market, risen, record, high, investors, display, growing, confidence, durability, countrys, economic, main, bovespa, index, sao, paolo, exchange, closed, points, friday, topping, previous, close, reached, day, buoyancy, reflects, optimism, economy, grow, much, brazil, recovering, last, years, recession, worst, decade, output, declined, president, luiz, inacio, lula, da, silva, elected, brazils, first, workingcl

In [40]:
def log_tf(d):
    
    doc   = d[0]
    terms = d[1]
    tf    = d[2]
    
    return(doc, terms, [1 + math.log(t, 10) for t in tf])

In [41]:
log_tf = tf.map(log_tf)

In [42]:
cols = ["Doc", "Terms", "Log-TF(t,d)"]
log_tf.toDF(cols).toPandas()

,Doc,Terms,"Log-TF(t,d)"
0,bsns_382.txt,"[ban, forced, retirement, employers, will, longer, able, force, workers, retire, unless, can, justify, it, government, announced, firms, barred, imposing, arbitrary, ages, new, european, age, discrimination, rules, default, introduced, permitted, request, staying, beyond, compulsory, although, right, refuse, trade, industry, secretary, patricia, hewitt, said, people, work, wanted, saying, statutory, free, continue, employing, long, competent, proposals, department, last, year, allowed, till, wished, business, leaders, opposed, plan, costly, cumbersome, british, chambers, commerce, welcomed, latest, proposal, this, move, today, best, worlds, ability, define, end, point, employeremployee, relationship, employees, flexibility, past, concern, cowardly, complete, uturn, makes, mockery, governments, socalled, commitment, outlawing, ageism, leaving, incoming, law, unravel, ...]","[1.0, 1.3010299956639813, 1.7781512503836434, 1.6020599913279623, 1.8450980400142567, 1.3010299956639813, 1.0, 1.0, 1.6020599913279623, 1.0, 1.0, 1.3010299956639813, 1.0, 1.3010299956639813, 1.3010299956639813, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.3010299956639813, 2.0791812460476247, 1.4771212547196624, 1.0, 1.3010299956639813, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.3010299956639813, 1.0, 1.3010299956639813, 1.0, 1.3010299956639813, 1.3010299956639813, 1.0, 1.0, 1.0, 1.8450980400142567, 1.4771212547196624, 1.4771212547196624, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, ...]"
1,bsns_050.txt,"[bad, weather, hits, nestle, sales, combination, rising, raw, material, costs, sluggish, european, economy, hit, swiss, food, drink, giant, revenue, dipped, bn, francs, ice, cream, mineral, water, dampened, wet, summer, however, nestles, profits, margins, helped, strong, performance, americas, china, raise, dividend, paying, back, debt, said, strength, franc, us, dollar, disposal, businesses, challenging, trading, conditions, europe, dented, poor, across, continent, contrast, prolonged, heat, wave, severely, affected, demand, bottled, also, fell, although, chocolate, coffee, frozen, goods, petcare, products, performed, better, elsewhere, enjoyed, exceptional, year, north, america, outperforming, market, terms, growth, added, strongly, africa, asia, despite, impact, high, oil, prices, political, instability, total, earnings, ...]","[1.3010299956639813, 1.3010299956639813, 1.0, 2.0, 1.7781512503836434, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.6020599913279623, 1.0, 1.0, 1.0, 1.0, 1.0, 1.6020599913279623, 1.3010299956639813, 1.3010299956639813, 1.3010299956639813, 1.3010299956639813, 1.4771212547196624, 1.0, 1.0, 1.3010299956639813, 1.0, 1.3010299956639813, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.3010299956639813, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.3010299956639813, ...]"
2,bsns_057.txt,"[economy, stronger, forecast, uk, probably, grew, faster, rate, third, quarter, reported, according, bank, england, deputy, governor, rachel, lomax, private, sector, business, surveys, suggest, official, estimates, ms, said, collectively, show, rapid, slowdown, house, price, growth, pointed, out, means, despite, strong, economic, base, rates, will, stay, hold, data, comes, office, national, statistics, ons, though, reliable, takes, longer, publish, now, boe, c

In [43]:
def df(t):
    
    term = t[0]
    dft  = t[1][0]
   
    return (term, dft)

In [44]:
df = inverted_index.map(df)

In [45]:
df.take(1)

[('retirement', 37)]

In [46]:
def idf(t):
    
    term = t[0]
    dft  = t[1]
    
    a = num_of_documents / dft
    idft = math.log(a, 10)
    
    return(term, idft)

In [47]:
idf = df.map(idf)

In [48]:
idf.take(1)

[('retirement', 1.779128291249955)]

In [49]:
idfMap = idf.collectAsMap()

In [50]:
idft = df.join(idf)

idft = idft.map(lambda t:(t[0], t[1][0], t[1][1]))

In [51]:
cols = ["Term", "DF(t)", "IDF(t)"]
idft.toDF(cols).toPandas()

,Term,DF(t),IDF(t)
0,retirement,37,1.779128
1,rules,114,1.290425
2,default,11,2.305937
3,introduced,67,1.521255
4,beyond,83,1.428252
...,...,...,...
31526,galaxies,1,3.347330
31527,sanity,1,3.347330
31528,succesfull,1,3.347330
31529,autolinks,1,3.347330


In [52]:
def tf_idf(d):
    
    doc   = d[0]
    terms = d[1]
    log_tf    = d[2]
    wtd = []
    
    for i in range(len(terms)):
        wtd.append(log_tf[i] * idfMap[terms[i]])
    
    return (doc, terms, wtd)

In [53]:
wtd = log_tf.map(tf_idf)

In [54]:
cols = ["Doc", "Terms", "W(td)= td-idf"]
wtd.toDF(cols).toPandas()

,Doc,Terms,W(td)= td-idf
0,bsns_382.txt,"[ban, forced, retirement, employers, will, longer, able, force, workers, retire, unless, can, justify, it, government, announced, firms, barred, imposing, arbitrary, ages, new, european, age, discrimination, rules, default, introduced, permitted, request, staying, beyond, compulsory, although, right, refuse, trade, industry, secretary, patricia, hewitt, said, people, work, wanted, saying, statutory, free, continue, employing, long, competent, proposals, department, last, year, allowed, till, wished, business, leaders, opposed, plan, costly, cumbersome, british, chambers, commerce, welcomed, latest, proposal, this, move, today, best, worlds, ability, define, end, point, employeremployee, relationship, employees, flexibility, past, concern, cowardly, complete, uturn, makes, mockery, governments, socalled, commitment, outlawing, ageism, leaving, incoming, law, unravel, ...]","[1.5479894658633686, 1.6546279011328127, 3.1635591956790226, 3.244348863455414, 0.33214104409515793, 1.7417276825556467, 0.9407898348829952, 1.302007036530293, 2.396796561215802, 2.0685764143641214, 1.5344166586740946, 0.6099335561941159, 2.305937330158725, 0.7022894699188306, 0.9081126355710337, 1.0292666803541888, 0.9744180123468438, 2.305937330158725, 3.180029593453413, 2.648360010980931, 2.1712387562612694, 0.35699116052934887, 1.1711507524074818, 2.8668729623814384, 3.406149042428964, 1.2904251639804776, 3.0000936346578184, 1.5212552126161238, 2.305937330158725, 2.5592805294942975, 2.5022319753026934, 1.4282519229408763, 2.8248467496440672, 0.9208187539523751, 1.132167700026648, 2.004907334494744, 1.487350543835994, 1.2559011327410157, 1.077817071099034, 2.305937330158725, 1.9493900066449126, 0.1316071997804136, 0.6716159875587938, 1.1105496763826253, 1.1220207335910874, 0.9373968919856557, 2.5022319753026934, 1.1654864273721777, 1.0528637891553574, 2.4442400283250065, 0.94249..."
1,bsns_050.txt,"[bad, weather, hits, nestle, sales, combination, rising, raw, material, costs, sluggish, european, economy, hit, swiss, food, drink, giant, revenue, dipped, bn, francs, ice, cream, mineral, water, dampened, wet, summer, however, nestles, profits, margins, helped, strong, performance, americas, china, raise, dividend, paying, back, debt, said, strength, franc, us, dollar, disposal, businesses, challenging, trading, conditions, europe, dented, poor, across, continent, contrast, prolonged, heat, wave, severely, affected, demand, bottled, also, fell, although, chocolate, coffee, frozen, goods, petcare, products, performed, better, elsewhere, enjoyed, exceptional, year, north, america, outperforming, market, terms, growth, added, strongly, africa, asia, despite, impact, high, oil, prices, political, instability, total, earnings, ...]","[1.764331902689513, 2.492729563436379, 1.466516423036159, 6.694660030633901, 1.8376531610613998, 1.915966251157963, 1.4608392901444682, 2.0920575102136443, 1.6941175015416063, 1.1626385844993514, 2.0685764143641214, 0.9001719839747311, 1.0617727063091766, 0.8775079993387873, 3.0441895121681943, 1.7138615597373636, 2.0685764143641214, 1.204315215062855, 1.541150041333063, 2.305937330158725, 1.3320426670715215, 3.5716786474068507, 3.0000936346578184, 3.3425786374011723, 3.963327701360288, 2.7211232876685876, 2.745270023988988, 2.648360010980931, 1.649899587783754, 0.7271939603431928, 4.354976755313726, 1.4230507292550685, 2.8248467496440672, 1.2504200023088938, 0.9856021792993573, 1.124613544169367, 1.9001719839747309, 1.45523541262647, 1.3516948207194002, 3.113478627395495, 1.6149362554939817, 0.7835521111244178, 1.4388449964383005, 0.09279990051820618, 1.6149362554939817, 3.046300019652969, 0.4230507292550687, 1.3742021617172517, 2.5691787649333064, 1.514821102610714, 1.93235666734..."
2,bsns_057.txt,"[economy, stronger, forecast, uk, probably, grew, faster, rate, third, quarter, reported, according, bank, england, deputy, governor, rachel, lomax, private, sector, business, surveys, suggest, of

In [55]:
# Save document as vector of tf-idf weights

# Format = {document: [[term in doc][tf-idf weights]]}

vecs = wtd.map(lambda d:(d[0], [d[1],d[2]]))

In [56]:
vecsMap = vecs.collectAsMap()

In [57]:
# Get the norm of a document vector

def get_length(doc_name:str):
    
    s= 0
    
    for w in vecsMap[doc_name][1]:
        s += math.pow(w, 2)
    
    return math.sqrt(s)

In [58]:
def initialize_length():
    
    length = {}
    
    for doc in vecsMap:
        length[doc] = get_length(doc)
        
    return length

In [71]:
# Cosine Score Algorithm

# parameters:
    # p_doc   = document name / null for a query
    # p_terms = terms of a document structured as Dict {term:num of appearances} 
    #           terms of a query structured as an array [term]
    # K       = top K closest documents 
    
def cosine_score(p_doc, p_terms, K): 
    
    N = num_of_documents
    scores = {}
    length = {}
    
    tf     = 0
    log_tf = 0
    dft    = 0
    idf    = 0
    wtd    = 0
    
    _tf     = 0
    _log_tf = 0
    _wtd    = 0
    
    inner_prod = 0
    
    length = initialize_length()
    
    for term in p_terms:
        
        if p_doc != "":
            tf = p_terms[term]
            log_tf = 1 + math.log(tf, 10)
        else:
            tf = 0
            log_tf = 0
            
        if term in broadcast_inverted_index.value:
            
            posting_list = broadcast_inverted_index.value[term] #inverted_indexMap[term]
            
            dft = posting_list[0]
            idf = math.log(N/dft, 10)
            wtd = tf * idf            
            
            for doc in posting_list[1]:
                 
                if doc != p_doc: 
                    
                    _tf     = posting_list[1][doc]
                    _log_tf = 1 + math.log(_tf, 10)
                    _wtd    = _log_tf * idf
                    
                    if p_doc != "":
                        inner_prod = wtd * _wtd
                    else:
                        inner_prod = _wtd
                        
                    if doc in scores:
                        scores[doc] += inner_prod
                    else:
                        scores[doc] = inner_prod
                    
    for doc in scores:
             
            scores[doc] = scores[doc] / length[doc]
             
    topK = sorted(scores.items(), key = operator.itemgetter(1), reverse = True)[0:K]
    
    return topK

In [72]:
def f_topK(d):
    
    doc   = d[0]
    terms = d[1]
    
    top = cosine_score(doc, terms, K)
    
    return (doc, top)

In [68]:
documents_with_cnt = documents.map(lambda d:(d[0], dict(Counter(d[1]))))

<h3>Match top K documents</h3>

Lets find for each document related top K similar documents

In [73]:
K = 5
topK = documents_with_cnt.map(f_topK)

In [74]:
topK.take(5)

[('bsns_382.txt',
  [('pltcs_132.txt', 7.048991438148983),
   ('pltcs_043.txt', 4.386427748684216),
   ('pltcs_294.txt', 4.083830732161672),
   ('pltcs_042.txt', 4.025764226928439),
   ('bsns_255.txt', 3.9326055985098076)]),
 ('bsns_050.txt',
  [('bsns_111.txt', 4.175012150937491),
   ('bsns_247.txt', 3.7323229627059518),
   ('bsns_173.txt', 3.6790900754115725),
   ('bsns_038.txt', 3.2569933231034094),
   ('bsns_143.txt', 2.936784643531735)]),
 ('bsns_057.txt',
  [('bsns_119.txt', 8.106474799713823),
   ('bsns_151.txt', 7.565008186070201),
   ('bsns_438.txt', 7.34675375397502),
   ('bsns_290.txt', 7.273170529606044),
   ('bsns_137.txt', 6.486337783622852)]),
 ('bsns_272.txt',
  [('bsns_305.txt', 7.541729091918997),
   ('bsns_490.txt', 7.5160479249769665),
   ('bsns_303.txt', 6.807061866587818),
   ('bsns_031.txt', 4.190471524226937),
   ('bsns_330.txt', 4.156201825997844)]),
 ('bsns_065.txt',
  [('bsns_075.txt', 17.06538754940708),
   ('bsns_109.txt', 5.021814870238858),
   ('bsns_466.

In [75]:
K = 1
top1 = documents_with_cnt.map(f_topK)

In [76]:
top1.take(5)

[('bsns_382.txt', [('pltcs_132.txt', 7.048991438148983)]),
 ('bsns_050.txt', [('bsns_111.txt', 4.175012150937491)]),
 ('bsns_057.txt', [('bsns_119.txt', 8.106474799713823)]),
 ('bsns_272.txt', [('bsns_305.txt', 7.541729091918997)]),
 ('bsns_065.txt', [('bsns_075.txt', 17.06538754940708)])]

In [77]:
def f_order_key(t):
    doc = t[0]
    top = t[1][0]
    score = top[1]
    
    return -1*score

In [78]:
top_top = top1.takeOrdered(5, f_order_key)

In [82]:
top_top

[('pltcs_293.txt', [('pltcs_111.txt', 75.9598216501749)]),
 ('tech_379.txt', [('tech_009.txt', 68.29190379758998)]),
 ('tech_009.txt', [('tech_379.txt', 58.14916607376993)]),
 ('tech_032.txt', [('tech_335.txt', 56.96882600223184)]),
 ('sprt_121.txt', [('sprt_112.txt', 56.012770615172414)])]

<h3>Match top K documents for a given query</h3>

As we catalog the collection of documents (for K-mean alogorithm) to different categories, 
We can check queries related to specific category and expect to get ducuments from that category 

In [83]:
def query_to_vec(q):
    vec = text_to_vec(("query", q))
    vec = remove_stop_words(vec)
    vec = remove_symbols(vec)
   
    return list(dict(Counter(vec[1])))

In [84]:
K =10

q1  = "Medal, Season, Jumpping Sport"
cs_q1 = cosine_score("",query_to_vec(q1), K)
cs_q1

[('sprt_001.txt', 0.23921802890231),
 ('sprt_074.txt', 0.22915904711488141),
 ('sprt_073.txt', 0.212012086302841),
 ('sprt_020.txt', 0.20404037207324605),
 ('sprt_012.txt', 0.20404037207324605),
 ('sprt_085.txt', 0.19727580656436186),
 ('sprt_360.txt', 0.17645403535557114),
 ('sprt_313.txt', 0.16953023104545495),
 ('sprt_071.txt', 0.16726169683410694),
 ('sprt_090.txt', 0.157500926384518)]

In [85]:
q2  = "Best Song or a movie or a show"
cs_q2 = cosine_score("",query_to_vec(q2), K)
cs_q2

[('entrtmnt_139.txt', 0.20965803308808034),
 ('entrtmnt_095.txt', 0.20008622764629388),
 ('entrtmnt_332.txt', 0.1899784886600477),
 ('entrtmnt_195.txt', 0.18646226183326164),
 ('entrtmnt_264.txt', 0.18496693980740608),
 ('entrtmnt_323.txt', 0.18111813984199393),
 ('entrtmnt_177.txt', 0.17120894548804563),
 ('entrtmnt_172.txt', 0.1699918591177055),
 ('entrtmnt_156.txt', 0.1677685594729802),
 ('entrtmnt_295.txt', 0.16490506314574613)]

In [86]:
q3  = "Parliament, Prime minister, Citizen"
cs_q3 = cosine_score("",query_to_vec(q3), K)
cs_q3

[('bsns_423.txt', 0.28332612680738756),
 ('pltcs_401.txt', 0.2488605320373799),
 ('pltcs_326.txt', 0.24209466201214577),
 ('pltcs_341.txt', 0.23137929732523377),
 ('pltcs_223.txt', 0.23137929732523377),
 ('pltcs_409.txt', 0.17872882272362992),
 ('bsns_106.txt', 0.17491794946119515),
 ('pltcs_417.txt', 0.1701159016949548),
 ('pltcs_225.txt', 0.1700946652185568),
 ('bsns_159.txt', 0.1673320286618806)]

In [87]:
q4  = "Stock market , my pension my money"
cs_q4 = cosine_score("",query_to_vec(q4), K)
cs_q4

[('bsns_206.txt', 0.2734247333066525),
 ('bsns_267.txt', 0.23249585865517133),
 ('bsns_274.txt', 0.23017317839139773),
 ('bsns_365.txt', 0.22871722230292169),
 ('bsns_233.txt', 0.2274119803553166),
 ('bsns_276.txt', 0.2058369651626627),
 ('bsns_255.txt', 0.19759770864611914),
 ('bsns_238.txt', 0.19220137562108702),
 ('bsns_281.txt', 0.19119795474638726),
 ('bsns_404.txt', 0.18935238829000345)]

In [88]:
q5  = "online software website"
cs_q5 = cosine_score("",query_to_vec(q5), K)
cs_q5

[('tech_284.txt', 0.20567159238928318),
 ('tech_036.txt', 0.19189804772402747),
 ('tech_003.txt', 0.19189804772402747),
 ('tech_369.txt', 0.17587356420411257),
 ('tech_278.txt', 0.16909032818754294),
 ('tech_259.txt', 0.15947519750057187),
 ('tech_040.txt', 0.1519206236148456),
 ('tech_332.txt', 0.1519206236148456),
 ('tech_194.txt', 0.14338066647445938),
 ('tech_149.txt', 0.138223796728688)]

Lets check what we can get when searching only one word:

In [89]:
q6  = "Retirement"
cs_q6 = cosine_score("",query_to_vec(q6), K)
cs_q6

[('bsns_255.txt', 0.17301923141337677),
 ('sprt_460.txt', 0.165253698857296),
 ('bsns_382.txt', 0.15737030857412618),
 ('pltcs_132.txt', 0.14112591621481335),
 ('entrtmnt_269.txt', 0.12477394874210603),
 ('sprt_347.txt', 0.12033074679621003),
 ('bsns_041.txt', 0.10426516380891843),
 ('sprt_241.txt', 0.10241755940575076),
 ('sprt_462.txt', 0.09775802608995864),
 ('pltcs_357.txt', 0.09224770028329213)]

The term "retirement" can be related to pension or to a sportsman who retired from the match.

Lets look in the results :

<b>Pension</b>:

 bsns_255.txt is about the UK state persion system<br>
 bsns_382.txt is about a new law which says that employers will no longer be able to force workers to retire before the age of 65<br>
 pltcs_132.txt is about a review of how the market will be influenced by raising the retirement age <br>
            
<b>Sports</b>

 sprt_460.txt is about Lindsay Devenport- American professional tennis player who retired from the match because   of a back injury<br>
 sprt_347.txt is mention Martin Osborne Johnson - an English retired rugby player<br>

 <b>Document frequency:</b>
 
 We can also notice that the term "retirement" is a rare term (dft = 37) , so even a document with tf = 2 like sprt_347.txt will be returned in the results set and it is actually very relevant

In [90]:
q7  = "A new song"
cs_q7 = cosine_score("",query_to_vec(q7), K)
cs_q7

[('entrtmnt_160.txt', 0.19107598611857668),
 ('entrtmnt_112.txt', 0.18027134506664644),
 ('entrtmnt_159.txt', 0.13977457860067408),
 ('entrtmnt_143.txt', 0.13536622854030908),
 ('entrtmnt_130.txt', 0.12543058260066214),
 ('entrtmnt_254.txt', 0.11991924485035944),
 ('entrtmnt_136.txt', 0.11921600720250274),
 ('entrtmnt_123.txt', 0.11755475166615598),
 ('entrtmnt_240.txt', 0.116455353918639),
 ('entrtmnt_156.txt', 0.11511913864915403)]

When searching for a new song we get the expected results:

entrtmnt_160.txt - the new singel "Band Aid" in iTunes<br>
entrtmnt_112.txt - the re-release of hit song of Elvis<br>
entrtmnt_159.txt - the new video of former Westlife

In [91]:
q8  = "Antivirus"
cs_q8 = cosine_score("",query_to_vec(q8), K)
cs_q8

[('tech_036.txt', 0.1646163806158948),
 ('tech_003.txt', 0.1646163806158948),
 ('tech_008.txt', 0.13660982036470645),
 ('tech_252.txt', 0.13660982036470645),
 ('tech_177.txt', 0.13501089969332708),
 ('tech_292.txt', 0.13501089969332708),
 ('tech_117.txt', 0.12325272038500676),
 ('tech_272.txt', 0.11881789795883863),
 ('tech_290.txt', 0.10830049849617138),
 ('tech_116.txt', 0.10800164318737573)]

When quering for the term "antivirus" we gets a usefull information about viruses and tools for example :

tech_036.txt- Microsoft spyware tool<br>
tech_003.txt - Windows virus disguising itself as an electronic Christmas card<br>
tech_117.txt - Joke e-mail virus tricks users

In [97]:
q9  = "Pythons"
cs_q9 = cosine_score("",query_to_vec(q9), K)
cs_q9

[('entrtmnt_290.txt', 0.16784961864690623)]

When searching for news about the language "Python" we did'nt find much - as expected.

The only document related is "entrtmnt_290.txt" with respct to "Monty Python"- British comedy troupe who created comedy tv shows

<b>Document frequency and term frequency:</b>

We should notice that although the term appears in the document only ones, it is the only document the term is exists in so of course, we are getting it in the result set 


In [96]:
inverted_indexMap["sex"]

(26,
 {'bsns_296.txt': 1,
  'bsns_336.txt': 1,
  'entrtmnt_246.txt': 1,
  'entrtmnt_377.txt': 1,
  'entrtmnt_381.txt': 1,
  'entrtmnt_199.txt': 1,
  'entrtmnt_040.txt': 1,
  'entrtmnt_201.txt': 1,
  'entrtmnt_035.txt': 1,
  'entrtmnt_268.txt': 1,
  'entrtmnt_281.txt': 1,
  'entrtmnt_196.txt': 1,
  'entrtmnt_203.txt': 1,
  'entrtmnt_168.txt': 5,
  'entrtmnt_176.txt': 3,
  'pltcs_286.txt': 1,
  'pltcs_244.txt': 2,
  'pltcs_266.txt': 1,
  'pltcs_288.txt': 1,
  'pltcs_003.txt': 1,
  'pltcs_125.txt': 1,
  'pltcs_371.txt': 1,
  'pltcs_393.txt': 1,
  'tech_002.txt': 1,
  'tech_362.txt': 2,
  'tech_245.txt': 1})

In [98]:
q10  = "Sex"
cs_q10 = cosine_score("",query_to_vec(q10), K)
cs_q10

[('entrtmnt_176.txt', 0.1316839628937215),
 ('entrtmnt_168.txt', 0.1257327736246843),
 ('entrtmnt_196.txt', 0.10761593710787999),
 ('entrtmnt_381.txt', 0.09602264048269268),
 ('entrtmnt_281.txt', 0.09283569784640852),
 ('entrtmnt_199.txt', 0.09265989071058442),
 ('entrtmnt_035.txt', 0.09077014633423246),
 ('tech_002.txt', 0.09060200560141587),
 ('pltcs_266.txt', 0.09050426744210363),
 ('pltcs_244.txt', 0.08899495990878042)]

When searching the term "sex" we are getting documents with low score- This is because for most, the term appears
in related documents only ones. 

In [100]:
q11  = "'Sex and the city' in HBO"
cs_q11 = cosine_score("",query_to_vec(q11), K)
cs_q11

[('entrtmnt_176.txt', 0.42619734985167507),
 ('entrtmnt_080.txt', 0.18690770137841886),
 ('entrtmnt_196.txt', 0.1786476130577306),
 ('entrtmnt_281.txt', 0.15411170754554704),
 ('entrtmnt_168.txt', 0.1257327736246843),
 ('bsns_070.txt', 0.11341810297682722),
 ('sprt_244.txt', 0.11131744875347242),
 ('sprt_206.txt', 0.11100880572569072),
 ('entrtmnt_201.txt', 0.10540749706861163),
 ('sprt_176.txt', 0.09988683506924169)]

When adding the terms "city" and "HBO" we are getting higher score with documents related to the tv show "sex and the city"

<h3>K-means algorithm</h3>



In [428]:
vecs.take(1)

[('bsns_382.txt',
  [['ban',
    'forced',
    'retirement',
    'employers',
    'longer',
    'able',
    'force',
    'workers',
    'retire',
    'unless',
    'justify',
    'government',
    'announced',
    'firms',
    'barred',
    'imposing',
    'arbitrary',
    'ages',
    'new',
    'european',
    'age',
    'discrimination',
    'rules',
    'default',
    'introduced',
    'permitted',
    'request',
    'staying',
    'beyond',
    'compulsory',
    'although',
    'right',
    'refuse',
    'trade',
    'industry',
    'secretary',
    'patricia',
    'hewitt',
    'said',
    'people',
    'work',
    'wanted',
    'saying',
    'statutory',
    'free',
    'continue',
    'employing',
    'long',
    'competent',
    'proposals',
    'department',
    'last',
    'year',
    'allowed',
    'till',
    'wished',
    'business',
    'leaders',
    'opposed',
    'plan',
    'costly',
    'cumbersome',
    'british',
    'chambers',
    'commerce',
    'welcomed',
    

In [276]:
def select_random_seeds():
    
    seeds = [] 
    
    while len(seeds) < K:
        
        seed = random.choice(list(vecsMap.keys()))
    
        if seed not in seeds:
            seeds.append(seed)
    
    return seeds

In [281]:
def get_euclidean_distance(doc1, doc2):
     
    ind1 = -1
    ind2 = -1
    ln = len(all_words)
    s = 0
    
    for i in range(ln):
        if all_words[i] in vecsMap[doc1][0] and all_words[i] in vecsMap[doc2][0]:
            ind1 = vecsMap[doc1][0].index(all_words[i])
            ind2 = vecsMap[doc2][0].index(all_words[i])
            s += math.pow(vecsMap[doc1][1][ind1] - vecsMap[doc2][1][ind2], 2)
            
        elif all_words[i] in vecsMap[doc1][0] and all_words[i] not in vecsMap[doc2][0]:
                ind1 = vecsMap[doc1][0].index(all_words[i])
                s += math.pow(vecsMap[doc1][1][ind1], 2)
        
        elif all_words[i] not in vecsMap[doc1][0] and all_words[i] in vecsMap[doc2][0]:
            ind2 = vecsMap[doc2][0].index(all_words[i])
            s += math.pow(vecsMap[doc2][1][ind2], 2)
        
    return math.sqrt(s)

In [254]:
print(get_euclidean_distance("bsns_382.txt", "bsns_383.txt"))

29.081117879422486


In [255]:
print(get_euclidean_distance("bsns_382.txt", "bsns_382.txt"))

0.0


In [277]:
K = 5

seeds = select_random_seeds()
seeds

['pltcs_355.txt',
 'pltcs_205.txt',
 'sprt_121.txt',
 'pltcs_104.txt',
 'tech_281.txt']

In [278]:
centroids = {}

for i in range(K):
    centroids[i] = seeds[i]

In [279]:
all_words = list(inverted_indexMap.keys())

In [287]:
def get_closest_center(v):
    
    doc = v[0]
    
    distance     = 0 
    min_distance = 0
    ret_val = 0
    
    min_distance = get_euclidean_distance(centroids[0], doc)

    for i in range(1, K):
        
        distance = get_euclidean_distance(centroids[i], doc)
        
        if min_distance > distance:
            min_distance = distance
            ret_val = i
    
    return (ret_val, min_val)

In [288]:
centroids_and_points =  vecs.map(get_closest_center)

In [ ]:
# Use combiners in order to get distance avarage

# t = tuple of (centroid , distance)
# return ( sum , count)

def create_combiner(t):
    return(t[1],1)

In [ ]:
def merge_value(accum, element):
    return (accum[0]+element[1], accum[1]+1)

In [ ]:
def merge_combiner(accum1, accum2):
    return (accum1[0] + accum2[0], accum1[1]+accum2[1])

In [ ]:
centroids_and_points.combinedByKey(create_combiner, merge_value, merge_combiner)

In [ ]:
def K_means(vecs, K):
    
    N = len(vecs)
    seeds = select_random_seeds()
    centroids = {}
    stop = 10
    j = -1
    wk = {}
   
    for i in range(K):
        centroids[i] = seeds[i]
        
        while stop > 0:
            for n in range(N):
                j = get_closest_center(vecs[n][1])
                if j in wk:
                    wk[j] = wk[j].append(n)
                else:
                    wk[j] = [n]
           
            for k in range(K):
               
                closest_vecs = wk[k]
                arr = np.array([])
               
                for cv in closest_vecs:
                    arr += np.array(vecs[cv])
                    
                    centroids[k] = arr / len(closest_vecs) 
             
            stop -= 1
       
        return wk

I have tried to implement K-means by regular map-reduce and combiners but I didnt get results 
I had a problem to comput the euclidean distance between vectos with a different dimensions
I have tried using numpy arrays and slso ParseVecotrs with no success
Finally I implemented the euclidean distance function in an inefficient way